## Network

In [ ]:
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
import torch.nn.functional as F
import torchvision.models as models

In [ ]:
class ImageEmbedding(nn.Module):
    def __init__(self,image_ftrs, pretrained = True): 
        super(ImageEmbedding, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        num_ftrs = self.resnet.fc.in_features #output 1000 from the resnet to be input to take the fc from the resnet output from doc
        self.resnet.fc = nn.Linear(num_ftrs, hidden_size) #numcls - is our own output classes
    
    def forward(self, image):
        image = self.resnet(image)    
        return image
    

In [ ]:
class QnsEmbedding(nn.Module):
    def __init__(self,input_size, question_ftrs, num_layers, batch_first = True):
        super(QnsEmbedding, self).__init__()
        self.lstm = nn.LSTM(input_size, question_ftrs, num_layers, batch_first = batch_first)
        
    def forward (self, inputs, cache):
        output, (hn, cn) = self.lstm(inputs, cache)
        return hn, cn
        

In [ ]:
#concat two models
class ConcatNet(nn.Module):
    def __init__(self):
        self.img_channel = ImageEmbedding(image_ftrs, pretrained = True)
        self.qns_channel = QnsEmbedding(input_size, question_ftrs, num_layers, batch_first = True)
        
        self.all = nn.Sequential(nn.Linear(numcl, 1024), 
                                nn.ReLU(), 
                                nn.Linear(1024, 3000), 
                                nn.Softmax(dim = 0))
#         self.fc1 = nn.Linear(numcl, 1024) 
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(1024, 3000)
#         self.softmax = nn.Softmax(dim = 0)
        
    def forward(self, image, questions):
        image_embed = self.img_channel(image) #returns tensor
        questions_embed = self.qns_channel(questions)
        added = torch.cat(image_embed,questions_embed)
        output = self.all(added)
        return output